In [39]:
import pandas as pd
from collections import Counter
from itertools import combinations, groupby, chain
import pickle
import warnings
warnings.filterwarnings('ignore')

In [5]:
aisles = pd.read_csv("aisles.csv")
departments = pd.read_csv("departments.csv")
order_products_prior = pd.read_csv("order_products__prior.csv")
order_products_train = pd.read_csv("order_products__train.csv")
orders = pd.read_csv("orders.csv")
products = pd.read_csv("products.csv")

In [6]:
products_aisles = pd.merge(products, aisles, on = 'aisle_id', how = 'left')
products_desc = pd.merge(products_aisles, departments, on = 'department_id', how = 'left')
products_desc.head()

,product_id,product_name,aisle_id,department_id,aisle,department
0,1,Chocolate Sandwich Cookies,61,19,cookies cakes,snacks
1,2,All-Seasons Salt,104,13,spices seasonings,pantry
2,3,Robust Golden Unsweetened Oolong Tea,94,7,tea,beverages
3,4,Smart Ones Classic Favorites Mini Rigatoni Wit...,38,1,frozen meals,frozen
4,5,Green Chile Anytime Sauce,5,13,marinades meat preparation,pantry


In [7]:
products_desc[products_desc['aisle']=='missing']

,product_id,product_name,aisle_id,department_id,aisle,department
37,38,Ultra Antibacterial Dish Liquid,100,21,missing,missing
71,72,Organic Honeycrisp Apples,100,21,missing,missing
109,110,Uncured Turkey Bologna,100,21,missing,missing
296,297,"Write Bros Ball Point Pens, Cap-Pen, Medium (1...",100,21,missing,missing
416,417,Classics Baby Binks Easter Chocolate Bunny,100,21,missing,missing
...,...,...,...,...,...,...
49552,49553,Organic Plain Skyr,100,21,missing,missing
49574,49575,Chocolate Vanilla Malt Custard,100,21,missing,missing
49640,49641,"8\"" Pecan Pie",100,21,missing,missing
49663,49664,Lemon Cayenne Drinking Vinegar,100,21,missing,missing


In [9]:
pickle.dump(products_desc, open("products_desc.p", "wb"))

In [10]:
orders_test = orders[orders['eval_set']=='test']
orders_test

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
38,2774568,3,test,13,5,15,11.0
44,329954,4,test,6,3,12,30.0
53,1528013,6,test,4,3,16,22.0
96,1376945,11,test,8,6,11,8.0
102,1356845,12,test,6,1,20,30.0
...,...,...,...,...,...,...,...
3420918,2728930,206202,test,23,2,17,6.0
3420929,350108,206204,test,5,4,14,14.0
3421001,1043943,206206,test,68,0,20,0.0
3421018,2821651,206207,test,17,2,13,14.0


In [11]:
merged_order_products = pd.merge(order_products_train, order_products_prior, how = 'outer')
merged_order_products.head()

,order_id,product_id,add_to_cart_order,reordered
0,1,49302,1,1
1,1,11109,2,1
2,1,10246,3,0
3,1,49683,4,0
4,1,43633,5,1


In [12]:
order_products_desc = pd.merge(merged_order_products, products_desc, on = 'product_id')
order_products_desc.head()

,order_id,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id,aisle,department
0,1,49302,1,1,Bulgarian Yogurt,120,16,yogurt,dairy eggs
1,816049,49302,7,1,Bulgarian Yogurt,120,16,yogurt,dairy eggs
2,1242203,49302,1,1,Bulgarian Yogurt,120,16,yogurt,dairy eggs
3,1383349,49302,11,1,Bulgarian Yogurt,120,16,yogurt,dairy eggs
4,1787378,49302,8,0,Bulgarian Yogurt,120,16,yogurt,dairy eggs


In [13]:
merged_orders = pd.merge(orders, order_products_desc, on = 'order_id')
merged_orders.head()

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id,aisle,department
0,2539329,1,prior,1,2,8,NaN,196,1,0,Soda,77,7,soft drinks,beverages
1,2539329,1,prior,1,2,8,NaN,26088,4,0,Aged White Cheddar Popcorn,23,19,popcorn jerky,snacks
2,2539329,1,prior,1,2,8,NaN,14084,2,0,Organic Unsweetened Vanilla Almond Milk,91,16,soy lactosefree,dairy eggs
3,2539329,1,prior,1,2,8,NaN,26405,5,0,XL Pick-A-Size Paper Towel Rolls,54,17,paper goods,household
4,2539329,1,prior,1,2,8,NaN,12427,3,0,Original Beef Jerky,23,19,popcorn jerky,snacks


In [14]:
test = orders[orders['eval_set']=='test']
test.head()

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
38,2774568,3,test,13,5,15,11.0
44,329954,4,test,6,3,12,30.0
53,1528013,6,test,4,3,16,22.0
96,1376945,11,test,8,6,11,8.0
102,1356845,12,test,6,1,20,30.0


In [16]:
pickle.dump(merged_orders, open("merged_orders.p", "wb"))

In [20]:
user_data = merged_orders[['user_id', 'order_number', 'order_dow', 'order_hour_of_day','days_since_prior_order', 'aisle']]

In [21]:
user_data = pd.get_dummies(user_data, prefix=None, columns=['aisle'])

In [22]:
# Splitting the data in thirds to use groupby then will rejoin them
user_data1 = user_data[user_data['user_id'] <= 65000]
user_data2 = user_data[(user_data['user_id'] <= 135000) & (user_data['user_id'] > 65000)]
user_data3 = user_data[user_data['user_id'] > 135000]

In [23]:
grouped_user1 = user_data1.groupby('user_id').sum()
grouped_user2 = user_data2.groupby('user_id').sum()
grouped_user3 = user_data3.groupby('user_id').sum()

In [24]:
user_data1_noaisles = user_data1.iloc[:,:5]
user_data2_noaisles = user_data2.iloc[:,:5]
user_data3_noaisles = user_data3.iloc[:,:5]

In [25]:
group1 = user_data1_noaisles.groupby('user_id').agg({'order_number': 'max', 'order_dow': lambda x:x.value_counts().index[0],
                                                     'order_hour_of_day': 'median', 'days_since_prior_order': 'mean'})
group2 = user_data2_noaisles.groupby('user_id').agg({'order_number': 'max', 'order_dow': lambda x:x.value_counts().index[0],
                                                     'order_hour_of_day': 'median', 'days_since_prior_order': 'mean'})
group3 = user_data3_noaisles.groupby('user_id').agg({'order_number': 'max', 'order_dow': lambda x:x.value_counts().index[0],
                                                     'order_hour_of_day': 'median', 'days_since_prior_order': 'mean'})

In [26]:
grouped_user1['num_orders'] = group1.order_number
grouped_user2['num_orders'] = group2.order_number
grouped_user3['num_orders'] = group3.order_number

grouped_user1['mean_days_since'] = group1.days_since_prior_order
grouped_user2['mean_days_since'] = group2.days_since_prior_order
grouped_user3['mean_days_since'] = group3.days_since_prior_order

grouped_user1['mode_order_dow'] = group1.order_dow
grouped_user2['mode_order_dow'] = group2.order_dow
grouped_user3['mode_order_dow'] = group3.order_dow

grouped_user1['median_order_hour'] = group1.order_hour_of_day
grouped_user2['median_order_hour'] = group2.order_hour_of_day
grouped_user3['median_order_hour'] = group3.order_hour_of_day

grouped_user1.drop(columns=['order_number', 'order_dow', 'order_hour_of_day', 'days_since_prior_order'], inplace=True)
grouped_user2.drop(columns=['order_number', 'order_dow', 'order_hour_of_day', 'days_since_prior_order'], inplace=True)
grouped_user3.drop(columns=['order_number', 'order_dow', 'order_hour_of_day', 'days_since_prior_order'], inplace=True)

grouped_users = pd.concat([grouped_user1, grouped_user2, grouped_user3], axis=0)
grouped_users.head()

,aisle_air fresheners candles,aisle_asian foods,aisle_baby accessories,aisle_baby bath body care,aisle_baby food formula,aisle_bakery desserts,aisle_baking ingredients,aisle_baking supplies decor,aisle_beauty,aisle_beers coolers,...,aisle_trail mix snack mix,aisle_trash bags liners,aisle_vitamins supplements,aisle_water seltzer sparkling water,aisle_white wines,aisle_yogurt,num_orders,mean_days_since,mode_order_dow,median_order_hour
user_id,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,2.0,11,19.200000,4,8.0
2,0.0,3.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,...,0.0,0.0,0.0,2.0,0.0,42.0,15,18.009390,1,10.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,2.0,0.0,0.0,12,11.487179,0,16.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,5,15.357143,4,13.0
5,1.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,3.0,5,12.314286,0,16.0


In [27]:
def movecol(df, cols_to_move=[], ref_col='', place='After'):

    cols = df.columns.tolist()
    if place == 'After':
        seg1 = cols[:list(cols).index(ref_col) + 1]
        seg2 = cols_to_move
    if place == 'Before':
        seg1 = cols[:list(cols).index(ref_col)]
        seg2 = cols_to_move + [ref_col]

    seg1 = [i for i in seg1 if i not in seg2]
    seg3 = [i for i in cols if i not in seg1 + seg2]

    return(df[seg1 + seg2 + seg3])

In [28]:
grouped_users = movecol(grouped_users,
                        cols_to_move=['num_orders', 'mode_order_dow', 'median_order_hour', 'mean_days_since'],
                        ref_col='aisle_air fresheners candles',
                        place='Before')
grouped_users.head()

,num_orders,mode_order_dow,median_order_hour,mean_days_since,aisle_air fresheners candles,aisle_asian foods,aisle_baby accessories,aisle_baby bath body care,aisle_baby food formula,aisle_bakery desserts,...,aisle_spreads,aisle_tea,aisle_tofu meat alternatives,aisle_tortillas flat bread,aisle_trail mix snack mix,aisle_trash bags liners,aisle_vitamins supplements,aisle_water seltzer sparkling water,aisle_white wines,aisle_yogurt
user_id,,,,,,,,,,,,,,,,,,,,,
1,11,4,8.0,19.200000,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0
2,15,1,10.0,18.009390,0.0,3.0,0.0,0.0,0.0,0.0,...,3.0,1.0,1.0,0.0,0.0,0.0,0.0,2.0,0.0,42.0
3,12,0,16.0,11.487179,0.0,0.0,0.0,0.0,0.0,0.0,...,4.0,1.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0
4,5,4,13.0,15.357143,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
5,5,0,16.0,12.314286,1.0,4.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0


In [29]:
pickle.dump(grouped_users, open("grouped_users.p", "wb"))

In [30]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaled_users = scaler.fit_transform(grouped_users)

In [32]:
from sklearn.cluster import KMeans
k_means_20 = KMeans(n_clusters=20, random_state=12, algorithm='full').fit(scaled_users)
pickle.dump(k_means_20, open("k_means_20.p", "wb"))

In [33]:
grouped_users['cluster'] = k_means_20.labels_

In [34]:
pickle.dump(grouped_users, open('clustered_users.p', 'wb'))

In [35]:
clustered_users = pickle.load(open("clustered_users.p", "rb"))

In [36]:
clustered_users.reset_index(inplace=True)

In [37]:
clustered_orders = merged_orders.merge(clustered_users[['user_id','cluster']], on = 'user_id')

In [38]:
def freq(iterable):
    if type(iterable) == pd.core.series.Series:
        return iterable.value_counts().rename("freq")
    else:
        return pd.Series(Counter(iterable)).rename("freq")


# Returns number of unique orders
def order_count(order_item):
    return len(set(order_item.index))

#Returns generator that yields item pairs, one at a time
def get_item_pairs(order_item):
    order_item = order_item.reset_index().to_numpy()
    for order_id, order_object in groupby(order_item, lambda x: x[0]):
        item_list = [item[1] for item in order_object]

        for item_pair in combinations(item_list, 2):
            yield item_pair


# Returns frequency and support associated with item
def merge_item_stats(item_pairs, item_stats):
    return (item_pairs
            .merge(item_stats.rename(columns={'freq': 'freqA', 'support': 'supportA'}),
                   left_on='item_A', right_index=True)
            .merge(item_stats.rename(columns={'freq': 'freqB', 'support': 'supportB'}),
                   left_on='item_B', right_index=True))


# Returns name associated with item
def merge_item_name(rules, item_name):
    columns = ['itemA','itemB','freqAB','supportAB','freqA','supportA','freqB','supportB',
               'confidenceAtoB','confidenceBtoA','lift']
    rules = (rules
             .merge(item_name.rename(columns={'item_name': 'itemA'}), left_on='item_A', right_on='item_id')
             .merge(item_name.rename(columns={'item_name': 'itemB'}), left_on='item_B', right_on='item_id'))

In [40]:
def association_rules(order_item, min_support):

    #     print("Starting number of order_item pairs: {:22d}".format(len(order_item)))
    print("Starting number of orders, items, order_item pairs: {}, {}, {}".format(order_item.index.nunique(),
                                                                                  order_item.nunique(),len(order_item)))

    # Calculate item frequency and support
    item_stats             = freq(order_item).to_frame("freq")
    item_stats['support']  = item_stats['freq'] / order_count(order_item)


    # Filter from order_item items below min support
    qualifying_items       = item_stats[item_stats['support'] >= min_support].index
    order_item             = order_item[order_item.isin(qualifying_items)]

    print("Items with support >= {}: {:15d}".format(min_support, len(qualifying_items)))
    #     print("Remaining order_item: {:21d}".format(len(order_item)))
    print("Remaning number of orders, items, order_item pairs: {}, {}, {}".format(order_item.index.nunique(),
                                                                                  order_item.nunique(),len(order_item)))


    # Filter from order_item orders with less than 2 items
    order_size             = freq(order_item.index)
    qualifying_orders      = order_size[order_size >= 2].index
    order_item             = order_item[order_item.index.isin(qualifying_orders)]

    print("Remaining orders with 2+ items: {:11d}".format(len(qualifying_orders)))
    #     print("Remaining order_item: {:21d}".format(len(order_item)))
    print("Remaining number of orders, items, order_item pairs: {}, {}, {}".format(order_item.index.nunique(),
                                                                                   order_item.nunique(),len(order_item)))


    # Recalculate item frequency and support
    item_stats             = freq(order_item).to_frame("freq")
    item_stats['support']  = item_stats['freq'] / order_count(order_item)


    # Get item pairs generator
    item_pair_gen          = get_item_pairs(order_item)


    # Calculate item pair frequency and support
    item_pairs              = freq(item_pair_gen).to_frame("freqAB")
    item_pairs['supportAB'] = item_pairs['freqAB'] / len(qualifying_orders)

    print("Item pairs: {:31d}".format(len(item_pairs)))


    # Filter from item_pairs those below min support
    item_pairs              = item_pairs[item_pairs['supportAB'] >= min_support]

    print("Item pairs with support >= {}: {:10d}\n".format(min_support, len(item_pairs)))


    # Create table of association rules and compute relevant metrics
    item_pairs = item_pairs.reset_index().rename(columns={'level_0': 'item_A', 'level_1': 'item_B'})
    item_pairs = merge_item_stats(item_pairs, item_stats)

    item_pairs['confAtoB'] = item_pairs['supportAB'] / item_pairs['supportA']
    item_pairs['confBtoA'] = item_pairs['supportAB'] / item_pairs['supportB']
    item_pairs['lift']           = item_pairs['supportAB'] / (item_pairs['supportA'] * item_pairs['supportB'])


    # Return association rules sorted by lift in descending order
    return item_pairs.sort_values('lift', ascending=False)

In [41]:
cluster_orders = {}
for x in range(0,20):
    i = clustered_orders[clustered_orders['cluster'] == x]
    i = i[['order_id', 'product_id']]
    i = i.set_index('order_id')['product_id'].rename('item_id')
    cluster_orders[x] = i

In [43]:
group_association_rules_dic = {}
for x in range(0,20):
    print('Cluster: ', x)
    group_association_rules_dic[x] = association_rules(cluster_orders[x],.0001)

Cluster:  0
Starting number of orders, items, order_item pairs: 22099, 14626, 264115
Items with support >= 0.0001:            8717
Remaning number of orders, items, order_item pairs: 22078, 8717, 256351
Remaining orders with 2+ items:       21637
Remaining number of orders, items, order_item pairs: 21637, 8717, 255910
Item pairs:                          808257
Item pairs with support >= 0.0001:     178869

Cluster:  1
Starting number of orders, items, order_item pairs: 785071, 46426, 7788280
Items with support >= 0.0001:           10650
Remaning number of orders, items, order_item pairs: 781265, 10650, 7187429
Remaining orders with 2+ items:      746198
Remaining number of orders, items, order_item pairs: 746198, 10650, 7152362
Item pairs:                        10064495
Item pairs with support >= 0.0001:      54507

Cluster:  2
Starting number of orders, items, order_item pairs: 53338, 16897, 754548
Items with support >= 0.0001:            8319
Remaning number of orders, items, order

In [44]:
# format item association rule dataframes and merge product names

for x in group_association_rules_dic:
    group_association_rules_dic[x] = group_association_rules_dic[x].merge(products_desc[['product_id','product_name']]
                                                                          .rename(columns = {'product_id':'item_A','product_name':'product_name_A'}), on = 'item_A')
    group_association_rules_dic[x] = group_association_rules_dic[x].merge(products_desc[['product_id','product_name']]
                                                                          .rename(columns = {'product_id':'item_B','product_name':'product_name_B'}), on = 'item_B')
    group_association_rules_dic[x] = group_association_rules_dic[x][['item_A','item_B','product_name_A',
                                                                     'product_name_B','freqAB','supportAB','freqA',
                                                                     'supportA','freqB','supportB','confAtoB',
                                                                     'confBtoA','lift']]

In [45]:
pickle.dump(group_association_rules_dic, open("grouped_association_rules_dic.p", "wb"))

In [81]:
def product_to_product(cluster, product_id, item_lift, product_name, num_products):
    df = group_association_rules_dic[cluster]
    df = df[(df['item_A'] == product_id) | (df['item_B'] == product_id)][['product_name_A','item_A','product_name_B','item_B','confAtoB','lift']].sort_values('lift', ascending = False)
    df = df[df['lift'] > item_lift]
    df = df.sort_values('lift', ascending = False)
    df = df.head(n = num_products)
    product_to_product_associations = df['product_name_A'].values.tolist()
    for x in df['product_name_B'].values.tolist():
        product_to_product_associations.append(x)
    product_to_product_associations = [x for x in product_to_product_associations if x != product_name]
    return product_to_product_associations

In [83]:
def product_recommender(user_id, product_id, item_lift, num_products):
    product_name = products.at[product_id -1,'product_name']
    cluster = clustered_users.at[user_id, 'cluster']
    return product_to_product(cluster = cluster, product_id = product_id, item_lift = item_lift,
                              product_name = product_name, num_2 = num_products)

In [84]:
for i in range(0,20):
    user = clustered_users[clustered_users['cluster']==i].sample().index[0]
    print('\ncluster', i, ': user ', user)
    print('\n'.join([x for x in (product_recommender(user, 39055, 1, 5))]))


cluster 0 : user  134922
Coffee House Blend Deep Roast Coffee K
Organic Frosted Wildberry Acai Toaster Pastries
Ham, Parma, Prosciutto di Parma
Birthday Cake
Salsa, Verde, Roasted

cluster 1 : user  152700
Thick & Crispy Tortilla Chips
Thin & Light Tortilla Chips
Original Hummus
Organic Hass Avocado
Limes

cluster 2 : user  39900
Super Soft Taco Flour Tortillas
Fit Popcorn Sea Salt
Roast Beef
Clear Plastic Wrap
Via Instant Dark French Roast Coffee

cluster 3 : user  112550
Thick & Crispy Tortilla Chips
Original Taco Seasoning Mix
Organic Avocado
Pure & Natural Sour Cream
Real Guacamole

cluster 4 : user  58385
Thin & Light Tortilla Chips
Thick & Crispy Tortilla Chips
Organic Large Brown Grade AA Cage Free Eggs
Organic Reduced Fat 2% Milk
Organic Whole String Cheese

cluster 5 : user  150674
Thin & Light Tortilla Chips
Guacamole Seasoning Mix
Diet Green Tea with Ginseng

cluster 6 : user  176430
Chocolate Mint Builder's Bar
Thick & Crispy Tortilla Chips
Real Guacamole
Mild Guacamole
Ma

In [86]:
product_recommender(22107, 13176, 1, 5)

['Lemon Verbena Hand Wash',
 'Edamame Hummus',
 'Croissants',
 'Cereal, Superfood, Acai Blueberry',
 'Organic Navy Beans No Salt Added Fat Free']